In [6]:
import pennylane as qml

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

import torch
import torch.nn as nn
from torch.autograd import Variable

In [7]:
n_samples = 1000
n_features = 8
X, Y = make_classification(n_samples=n_samples, n_features=n_features)

x = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(Y.astype(np.float32)).view(-1,1)

In [38]:
n_qubits = 3
q_depth = 4
dev = qml.device("lightning.qubit", wires = n_qubits)

def angle_embedding(inputs):
    for i in range(n_qubits):
        qml.Hadamard(wires = i)
        
    for i in range(n_qubits):
        qml.RY(inputs[(2 * i) ], wires=i)
        qml.RZ(inputs[(2 * i) + 1], wires=i)

def amplitude_embedding(inputs):
    amp = 1/(1+torch.exp(-inputs))
    amp = amp/torch.sqrt(sum(amp**2))
    qml.StatePrep(amp, wires=range(3))

embedding = amplitude_embedding

@qml.qnode(dev, interface='torch')
def qnode(inputs, weights):

    embedding(inputs)

    for i in range(q_depth):
        for j in range(n_qubits):
            qml.RY(weights[2*(i*n_qubits + j) ], wires=j)
            qml.RZ(weights[2*(i*n_qubits + j) + 1], wires=j)
        for j in range(n_qubits):
            if (j == (n_qubits - 1)):
                qml.CNOT(wires=[j,0])
            else:
                qml.CNOT(wires=[j,j+1])

    for i in range(n_qubits):
        qml.RY(weights[(2*q_depth * n_qubits) + i ], wires=i)
        qml.RZ(weights[(2*q_depth * n_qubits) + i + 1], wires=i)

    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

In [39]:
n_args = 2*(q_depth +1) * n_qubits
weight_shapes = {"weights": n_args}

class QNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_features, 8)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.qlayer_1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.relu2 = nn.ReLU()
        self.out = nn.Linear(3, 1)
        self.out_act = nn.Sigmoid()

    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        a2 = self.qlayer_1(dout[0])
        h2 = self.relu2(a2.unsqueeze(0))
        a3 = self.out(h2)
        y = self.out_act(a3)
        return y

qnet = QNet()

In [40]:
opt = torch.optim.Adam(qnet.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()

num_epochs = 10

qnet.train()

e_losses = []
for e in range(num_epochs):
    losses = []
    for i in range(0, x.size(0)):
        x_batch = x[i:i+1, :]
        y_batch = y[i:i+1, :]
        x_batch = Variable(x_batch)
        y_batch = Variable(y_batch)

        opt.zero_grad()
        y_hat = qnet(x_batch)
        loss = criterion(y_hat, y_batch)
        loss.backward()
        opt.step()
        losses.append(loss.data.numpy())
    print(sum(losses)) 
    e_losses += losses


633.4203040599823
533.805628567934
421.1389866620302
346.5981124192476
292.9769009500742
268.1575214639306
254.4173020236194
241.57939662784338
234.21226861327887
226.9290564097464


In [41]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

test_pred_q = torch.empty((1000,1))
for i in range(0, x.size(0)):
    x_temp = x[i:i + 1, :]
    test_pred_q[i] = torch.round(qnet(x_temp))
accuracy_fn(y, test_pred_q)

92.4

| embedding | accuracy |
|---|---|
| angle | 92.1% |
|  amplitude | 92.4% |
| base | ??? |